In [ ]:
import os

def search_dataset(dataset, search_terms, search_type, bool_oper='or'):

    # Validate input parameters
    if not search_terms:
        raise ValueError("Missing search term(s)")

    if not search_type:
        raise ValueError("Missing search type (meta or text)")

    # Define dataset location (can replace this with global variables)
    dataset_folders = {
        1: "EEBO",
        2: "MEDICI",
        3: "EVANS",
        4: "Test Group"
    }

    # Set folder path for search based on search_type
    folder_path = os.path.join(os.getcwd(), "..", dataset_folders[dataset], search_type)

    # full path to verify
    full_path = os.path.abspath(folder_path)

    # Check if the path exists
    if not os.path.exists(full_path):
        raise FileNotFoundError(f"The system cannot find the path specified: {full_path}")

    # Create a function to load documents (text or meta)
    # Will change depending on implementation: either reading from disk or querying a database
    def load_documents():
        documents = []
        for doc_file in os.listdir(full_path):
            with open(os.path.join(full_path, doc_file), 'r', encoding='utf-8') as f:
                documents.append(f.read())
        return documents

    # Extract title from document
    def extract_title(doc):
        title_start = doc.find("TITLE:")
        if title_start != -1:
            title_end = doc.find("\n", title_start)
            return doc[title_start:title_end].strip()
        return "Unknown Title"

    # Perform search
    def perform_search(documents):
        results = []
        lower_search_terms = [term.lower() for term in search_terms]
        for doc in documents:
            lower_doc = doc.lower()
            if bool_oper == 'and':
                if all(term in lower_doc for term in lower_search_terms):
                    results.append(extract_title(doc))
            elif bool_oper == 'or':
                if any(term in lower_doc for term in lower_search_terms):
                    results.append(extract_title(doc))
        if len(results) == 0:
            return "No documents found with specified text"
        return results

    # Load and search documents
    documents = load_documents()
    search_results = perform_search(documents)

    return search_results

# Example usage
print(search_dataset(4, ["anthony"], "Text", "and"))
